In [3]:
# Vizuelna podešavanja okruženja (samo razvuče notebook na širinu ekrana)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))


In [6]:
import networkx as nx
import pickle

with open('data_cleaned/data2019_cleaned', 'rb') as file:
    data = pickle.load(file)
print(data)

                           tourney_id                  tourney_name surface  \
0                           2019-M020                      Brisbane    Hard   
1                           2019-M020                      Brisbane    Hard   
2                           2019-M020                      Brisbane    Hard   
3                           2019-M020                      Brisbane    Hard   
4                           2019-M020                      Brisbane    Hard   
...                               ...                           ...     ...   
2776  2019-M-DC-2019-QLS-M-SUI-RUS-01  Davis Cup QLS R1: SUI vs RUS    Hard   
2777  2019-M-DC-2019-QLS-M-SUI-RUS-01  Davis Cup QLS R1: SUI vs RUS    Hard   
2778  2019-M-DC-2019-QLS-M-SWE-COL-01  Davis Cup QLS R1: SWE vs COL    Clay   
2779  2019-M-DC-2019-QLS-M-SWE-COL-01  Davis Cup QLS R1: SWE vs COL    Clay   
2780  2019-M-DC-2019-QLS-M-SWE-COL-01  Davis Cup QLS R1: SWE vs COL    Clay   

      draw_size tourney_level  tourney_date  match_

In [7]:
lista_winners = set(data['winner_id'].unique())
lista_losers = set(data['loser_id'].unique())

svi_igraci = lista_winners.union(lista_losers)
print(f"Ukupan broj igraca je {len(svi_igraci)}, a ukupan broj meceva je {data.shape[0]}")


Ukupan broj igraca je 364, a ukupan broj meceva je 2781


In [15]:
import pandas as pd

dataPath = 'data/atp_players.csv'
players = pd.read_csv(dataPath)

dataPath = 'data/atp_rankings_10s.csv'
rankings = pd.read_csv(dataPath)

In [16]:
G = nx.Graph()
for player_id in svi_igraci:
    player = players[players['player_id'] == player_id]
    if player.empty:
        print("Postoji id igraca koji ne postoji u listi svih igraca!")
    else:
        player_rank = rankings[(rankings['player'] == player_id) & (rankings['ranking_date'] > 20190000) & (rankings['ranking_date'] < 20200000)]
        rank_p = 0
        points_p = 0
        
        if not player_rank.empty:
            last_row = player_rank.shape[0] - 1
            rank_p = int(player_rank.iloc[last_row, 1])
            points_p = int(player_rank.iloc[last_row, 3])
            
        G.add_node(str(player_id), first_name = str(player.iloc[0, 1]), last_name = str(player.iloc[0, 2]), country = str(player.iloc[0, 5]), hand = str(player.iloc[0, 3]), rank = rank_p, points = points_p)


In [17]:
data_reduced = data[['winner_id', 'loser_id']]
data_reduced = data_reduced.astype({'winner_id': str, 'loser_id': str})

for _, winner_id, loser_id in data_reduced.itertuples():
    if (winner_id, loser_id) in G.edges:
        G.edges[winner_id, loser_id]['weight'] += 1
    else:
        G.add_edge(winner_id, loser_id, weight = 1)

In [18]:
output_path = "models/model2019.gml"
nx.write_gml(G, output_path)

In [19]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 364
Number of edges: 2378
Average degree:  13.0659
